In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, Tuple
import math

@dataclass
class LLMconfig():
    # MHA
    hidden_size: int
    num_heads: int
    max_position_embeddings: int
    rope_theta: float
    
    attention_dropout: float
    

    # MLA
    q_lora_dim: int 
    kv_lora_dim: int
    v_head_dim: int
    qk_nope_head_dim: int
    qk_rope_head_dim: int
    attention_bias: bool
    
    

In [58]:
class DeepseekV2RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)


class DeepseekV2RotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()

        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base
        inv_freq = 1.0 / (
            self.base ** (torch.arange(0, self.dim, 2).float().to(device) / self.dim)
        )
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        # 较小索引位置对应较低频率
        # 较大的索引位置有较高的频率

        # Build here to make `torch.jit.trace` work.
        self._set_cos_sin_cache(
            seq_len=max_position_embeddings,
            device=self.inv_freq.device,
            dtype=torch.get_default_dtype(),
        )
        self.max_seq_len_cached = max_position_embeddings

    def _set_cos_sin_cache(self, seq_len, device, dtype):
        self.max_seq_len_cached = seq_len
        t = torch.arange(
            self.max_seq_len_cached, device=device, dtype=self.inv_freq.dtype
        )

        freqs = torch.outer(t, self.inv_freq.to(t.device))
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos().to(dtype), persistent=False)
        self.register_buffer("sin_cached", emb.sin().to(dtype), persistent=False)

    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        if seq_len is not None and seq_len > self.max_seq_len_cached:
            print("seq_len:", seq_len, self.max_seq_len_cached)
            self._set_cos_sin_cache(seq_len=seq_len, device=x.device, dtype=x.dtype)

        return (
            self.cos_cached[:seq_len].to(dtype=x.dtype),
            self.sin_cached[:seq_len].to(dtype=x.dtype),
        )


# Copied from transformers.models.llama.modeling_llama.rotate_half
def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


# Copied from transformers.models.llama.modeling_llama.apply_rotary_pos_emb
def apply_rotary_pos_emb(q, k, cos, sin, position_ids, unsqueeze_dim=1):
    cos = cos[position_ids].unsqueeze(unsqueeze_dim)
    sin = sin[position_ids].unsqueeze(unsqueeze_dim)

    b, h, s, d = q.shape
    q = q.view(b, h, s, d // 2, 2).transpose(4, 3).reshape(b, h, s, d)

    b, h, s, d = k.shape
    k = k.view(b, h, s, d // 2, 2).transpose(4, 3).reshape(b, h, s, d)

    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


# Copied from transformers.models.llama.modeling_llama.apply_rotary_pos_emb
def apply_rotary_pos_emb_v2(q: torch.Tensor, cos, sin, position_ids, unsqueeze_dim=1):
    cos = cos[position_ids].unsqueeze(unsqueeze_dim)
    sin = sin[position_ids].unsqueeze(unsqueeze_dim)

    b, h, s, d = q.shape
    q = q.view(b, h, s, d // 2, 2).transpose(4, 3).reshape(b, h, s, d)

    q_embed = (q * cos) + (rotate_half(q) * sin)
    return q_embed

In [59]:



class MLAv2(nn.Module):
    def __init__(self, config: LLMconfig):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_heads
        self.head_dim = self.hidden_size // self.num_heads
        # 定义
        self.qk_head_dim = config.qk_nope_head_dim + config.qk_rope_head_dim
        self.v_head_dim = config.v_head_dim
        self.qk_nope_head_dim = config.qk_nope_head_dim
        self.qk_rope_head_dim = config.qk_rope_head_dim
        # q的压缩和分片
        self.q_down_proj = nn.Linear(
            self.hidden_size,
            config.q_lora_dim,
            bias=False
        )

        self.q_up_proj = nn.Linear(
            config.q_lora_dim,
            self.num_heads * (self.qk_nope_head_dim + self.qk_rope_head_dim)
            ,
            bias=False
        )
        #q的lora
        self.q_lora_layernorm = DeepseekV2RMSNorm(config.q_lora_dim)
        
        #kv的压缩和分片
        self.kv_lora_dim = config.kv_lora_dim
        self.kv_layernorm = DeepseekV2RMSNorm(config.kv_lora_dim)
        self.kv_down_proj = nn.Linear(
            self.hidden_size,
            config.kv_lora_dim + config.qk_rope_head_dim,
            bias= False
        )
        self.kv_up_proj = nn.Linear(
            self.kv_lora_dim,
            self.num_heads * (self.v_head_dim + self.qk_head_dim - self.qk_rope_head_dim),
            bias= False
        )
        #rope部分
        self.rope_theta = config.rope_theta
        self.max_position_embeddings = config.max_position_embeddings
        self.rope = DeepseekV2RotaryEmbedding(
            self.qk_rope_head_dim,
            max_position_embeddings=self.max_position_embeddings,
            base=self.rope_theta
        )
        #out_proj
        self.out_proj = nn.Linear(
            self.num_heads * self.v_head_dim,
            self.hidden_size,
            bias=False
        )

    def forward(self, x, seq_len=None):#x:[bah_sze, seq_l, hddeim]if maxsqlecac is None or seq_len > self.max_seqeleu_cachnd:opout(config.attention_dropout)
        self.v_head_dim = self.v_head_dim
        self.out_proj = nn.Linear(
            self.num_heads * self.v_head_dim,
            self.hidden_size,
            bias=False
        )


    def forward(
            self, 
            hidden_states: torch.Tensor,
            position_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,  
            compress_kv: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor, torch.Tensor]]]:
        
        # 1.q的压缩和分片
        bsz, seq_len, _ = hidden_states.shape
        q = self.q_up_proj(self.q_down_proj(hidden_states)) #q:[bsz, seq_len, num_heads, qk_head_dim]
        q = q.view(bsz, seq_len, self.num_heads, self.qk_head_dim).transpose(1, 2) #q:[bsz, num_heads, seq_len, qk_head_dim]
        q_nope, q_pos = q.split(
            [self.qk_nope_head_dim, self.qk_rope_head_dim],
            dim=-1
        )
        # q_nope:[bsz, num_heads, seq_len, qk_nope_head_dim]
        # q_pos:[bsz, num_heads, seq_len, qk_rope_head_dim]

        # 2.kv的压缩和分片, compress_kv是入参
        kv_seq_len = compress_kv.size(1) # size（1） 把compress_kv的第二维拿出来,赋值给kv_seq_len
        compress_kv, k_pos = torch.split(
            compress_kv,
            [
                self.kv_lora_dim,
                self.qk_rope_head_dim
            ],
            dim=-1
        )#compress_kv:[bsz, kv_seq_len, kv_lora_dim + qk_rope_head_dim]
        # k_pos:[bsz, kv_seq_len, qk_rope_head_dim]

        # 3.kv的up_proj
        # 这里的k_pos长度是kv_seq_len,原来是seq_len
        print(f"Original weight shape: {self.kv_up_proj.weight.shape}")
        k_pos = k_pos.view(bsz, kv_seq_len, 1, self.qk_rope_head_dim).transpose(1, 2)
        print('k_pos shape:', k_pos.shape)
        # k_pos:[bsz, 1, kv_seq_len, qk_rope_head_dim]
        kv_up_proj = self.kv_up_proj.weight.view(self.num_heads, -1, self.kv_lora_dim)
        print(f"kv_up_proj shape: {kv_up_proj.shape}")
        # kv_up_proj:[num_heads, (), kv_lora_dim]

        q_absorb = kv_up_proj[:, :self.qk_nope_head_dim, :]
        # q_absorb:[num_heads, qk_nope_head_dim, kv_lora_dim]
        print('q_absorb shape:', q_absorb.shape)
        out_absorb = kv_up_proj[:, self.qk_nope_head_dim:, :]
        # out_absorb:[num_heads, qk_rope_head_dim, kv_lora_dim]
        print('out_absorb shape:', out_absorb.shape)
        # 4.应用rope
        cos, sin = self.rope(q_pos)
        q_pos = apply_rotary_pos_emb_v2(
            q_pos,
            cos,
            sin,
            position_ids=position_ids
        )
        print('q_pos shape:', q_pos.shape)

        # q_nope:[bsz, num_heads, seq_len, qk_nope_head_dim]
        # q_absorb:[num_heads, qk_nope_head_dim, kv_lora_dim]
        q_nope = torch.matmul(q_nope, q_absorb)
        print('q_nope shape:', q_nope.shape)
        # q_nope shape: (bsz, num_heads, seq_len, kv_lora_dim)
        # q_pos shape: (bsz, num_heads, seq_len, qk_rope_head_dim)
        # k_pos shape: (bsz, num_heads, kv_seq_len, qk_rope_head_dim)
        attn_weights = (
            torch.matmul(q_pos, k_pos.mT) + torch.matmul(q_nope, compress_kv.unsqueeze(-3).mT)
        ) / math.sqrt(self.qk_head_dim) #mT是转置最后两个维度
        print('attn_weights shape:', attn_weights.shape)
        # attn_weights shape:(bsz, seq_len, 1, kv_seq_len)
        attn_weights = F.softmax(attn_weights, dim=-1, dtype = torch.float32)
        # attn_weights shape: (bsz, seq_len, 1, kv_seq_len)
        # compress_kv shape: (bsz, kv_seq_len, kv_lora_dim)
        # b = bsz, s = seq_len, l = kv_seq_len, h = num_heads, q = qk_nope_head_dim, c = kv_lora_dim
        attn_out = torch.einsum('bsal, blc -> bsac', attn_weights, compress_kv)
        print('attn_out shape:', attn_out.shape)
        # attn_out shape: (bsz, seq_len, num_heads, kv_lora_dim)
        # out_absorb shape: (num_heads, qk_rope_head_dim, kv_lora_dim)
        attn_out = torch.matmul(attn_out, out_absorb.mT)
        # attn_out shape: (bsz, seq_len, num_heads, qk_rope_head_dim)
        attn_out = attn_out.transpose(1, 2).reshape(bsz, seq_len, -1)
        attn_out = self.out_proj(attn_out)
        print('attn_out shape:', attn_out.shape)
        return attn_out, attn_weights












        


In [60]:
# 写一个测试函数
def test_mlav2():
    config = LLMconfig(
        hidden_size=7168,
        num_heads=16,
        max_position_embeddings=1024,
        rope_theta=128000,
        attention_dropout=0.1,
        q_lora_dim=1536,
        qk_rope_head_dim=64,
        kv_lora_dim=512,
        v_head_dim=128,
        qk_nope_head_dim=128,
        attention_bias=False,
    )
    # 测试 MLAv2 attention
    # 生成 compressed_kv 的步骤:
    # 1. 先生成原始的 kv hidden states, shape 是 [bsz, kv_seq_len, hidden_size]
    # 2. 用 kv_a_proj_with_mqa 投影到 [bsz, kv_seq_len, kv_lora_rank + qk_rope_head_dim]
    # 3. split 成 compressed_kv 和 k_pe 两部分
    # 4. compressed_kv 过 layernorm
    # 5. k_pe 过 RoPE
    # 6. 最后把 compressed_kv 和 k_pe concat 在一起

    bsz = 2
    q_len = 1
    kv_seq_len = 12
    hidden_size = config.hidden_size

    # 生成测试数据
    q = torch.randn(bsz, q_len, hidden_size).cuda()
    # position_ids = torch.arange(q_len).expand(bsz, -1).cuda()
    position_ids = torch.full((bsz, q_len), 12, dtype=torch.long).cuda()

    # 初始化模型
    model = MLAv2(config).cuda()

    # 先随机初始化一个 compressed_kv
    compressed_kv = torch.randn(
        bsz, kv_seq_len, config.kv_lora_dim + config.qk_rope_head_dim
    ).cuda()
    print(
        "compressed_kv shape:", compressed_kv.shape
    )  # [bsz, kv_seq_len, kv_lora_dim + qk_rope_head_dim]

    # 前向计算
    output, attn_weights = model(q, position_ids, None, compressed_kv)
    print("output shape:", output.shape)  # [bsz, q_len, hidden_size]
    print(
        "attn_weights shape:", attn_weights.shape
    )  # [bsz, num_heads, q_len, kv_seq_len]


test_mlav2()

compressed_kv shape: torch.Size([2, 12, 576])
Original weight shape: torch.Size([4096, 512])
k_pos shape: torch.Size([2, 1, 12, 64])
kv_up_proj shape: torch.Size([16, 256, 512])
q_absorb shape: torch.Size([16, 128, 512])
out_absorb shape: torch.Size([16, 128, 512])
q_pos shape: torch.Size([2, 16, 1, 64])
q_nope shape: torch.Size([2, 16, 1, 512])
attn_weights shape: torch.Size([2, 16, 1, 12])
attn_out shape: torch.Size([2, 16, 1, 512])
attn_out shape: torch.Size([2, 1, 7168])
output shape: torch.Size([2, 1, 7168])
attn_weights shape: torch.Size([2, 16, 1, 12])
